# Tutorial 1: Getting Started with SQuADDS

In this tutorial, we will walk you through some basic usage of SQuADDS. By the end of this tutorial, you will be able to:

- Have an HuggingFace account

- Access the SQuADDS Database

- Use the SQuADDS API to query for closest and "best-guess" interpolated device designs for your chosen Hamiltonian parameters

- Simulate the "best-guess" design using an EM solver tool

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip install -e ../.

Obtaining file:///Users/shanto/LFL/SQuADDS/SQuADDS
  Preparing metadata (setup.py) ... done
  Attempting uninstall: SQuADDS
    Found existing installation: SQuADDS 0.0.1
    Uninstalling SQuADDS-0.0.1:
      Successfully uninstalled SQuADDS-0.0.1
  Running setup.py develop for SQuADDS


Since the SQuADDS Database is hosted on [HuggingFace](https://huggingface.co/), we will need to create an account and get an API key to access the database.

## HuggingFace 🤗 

HuggingFace is a company that provides a large number of NLP models and datasets. They also provide a platform to host your own models and datasets.

### Creating an  Account

Follow the instructions here - [HuggingFace: Sign Up](https://huggingface.co/join) - to create an account.

Once you have created an account, you can get your API key from the [settings page](https://huggingface.co/settings/token).

Please update the `HUGGINGFACE_API_KEY` variable in the `.env` file with your API key or execute the following code to set the environment variable.

In [ ]:
from squadds.core.utils import set_huggingface_api_key

set_huggingface_api_key()

### Login

To login to your HuggingFace account, run the following command in your terminal:

```bash
huggingface-cli login
```

You will be prompted to enter your username and password. Once you have logged in, you can check your login status by running the following command:

```bash
huggingface-cli whoami
```


### Accessing the SQuADDS Database using the HuggingFace API

The SQuADDS Database is hosted on HuggingFace. You can access the database using the `datasets` library from HuggingFace.

In [ ]:
from datasets import get_dataset_config_names
from datasets import load_dataset

configs = get_dataset_config_names("SQuADDS/SQuADDS_DB")

In [ ]:
components = []
component_names = []
data_types = []

for config in configs:
    components.append(config.split("-")[0])
    component_names.append(config.split("-")[1])
    data_types.append(config.split("-")[2])
    
print(components)
print(component_names)
print(data_types)
    

In [ ]:
qubit_data = load_dataset("SQuADDS/SQuADDS_DB", configs[0])
qubit_data

## Using the SQuADDS API to access and anlyze the database

While it is possible to directly access the SQuADDS Database using the `datasets` library, we have created a simple API to make it easier to query the database.

### Making Systems out of Circuit QED Elements

In [53]:
from squadds import SQuADDS_DB

In [54]:
db = SQuADDS_DB()

In [5]:
db.view_datasets()

╒═════════════╤══════════════════╤══════════════════╕
│ Component   │ Component Name   │ Data Available   │
╞═════════════╪══════════════════╪══════════════════╡
│ qubit       │ TransmonCross    │ cap_matrix       │
├─────────────┼──────────────────┼──────────────────┤
│ cavity_claw │ RouteMeander     │ eigenmode        │
├─────────────┼──────────────────┼──────────────────┤
│ coupler     │ NCap             │ cap_matrix       │
╘═════════════╧══════════════════╧══════════════════╛


In [6]:
db.get_dataset_info(component="qubit", component_name="TransmonCross", data_type="cap_matrix")

Dataset Features:
{'contributor': {'PI': Value(dtype='string', id=None),
                 'date_created': Value(dtype='string', id=None),
                 'group': Value(dtype='string', id=None),
                 'institution': Value(dtype='string', id=None),
                 'uploader': Value(dtype='string', id=None)},
 'design': {'design_options': {...},
            'design_tool': Value(dtype='string', id=None)},
 'notes': {},
 'sim_options': {'renderer_options': {...},
                 'setup': {...},
                 'simulator': Value(dtype='string', id=None)},
 'sim_results': {'claw_to_claw': Value(dtype='float64', id=None),
                 'claw_to_ground': Value(dtype='float64', id=None),
                 'cross_to_claw': Value(dtype='float64', id=None),
                 'cross_to_cross': Value(dtype='float64', id=None),
                 'cross_to_ground': Value(dtype='float64', id=None),
                 'ground_to_ground': Value(dtype='float64', id=None),
                 'u

In [ ]:
db.get_dataset_info(component="cavity_claw", component_name="RouteMeander", data_type="eigenmode")

In [ ]:
db.get_dataset(component="qubit", component_name="TransmonCross", data_type="cap_matrix")

In [ ]:
db.get_configs()

In [ ]:
db.view_contributors_of("qubit", "TransmonCross", "cap_matrix")

In [ ]:
db.view_all_contributors()

### Querying for the a target qubit design

In [70]:
db.select_system("qubit")

In [71]:
db.select_qubit("TransmonCross")

In [72]:
db.selected_system

'qubit'

In [73]:
df = db.selected_system_df()
df

,claw_to_claw,claw_to_ground,cross_to_claw,cross_to_cross,cross_to_ground,ground_to_ground,units,PI,date_created,group,institution,uploader,renderer_options,setup,simulator,design_options,design_tool
0,94.97421,90.86585,3.73363,158.40783,158.40783,311.25590,nH,"Eli Levenson-Falk, PhD",2023-09-20-142547,LFL,USC,Andre Kuo,"{'Cj': 0, 'Lj': '10nH', '_Rj': 0, 'design_name...","{'auto_increase_solution_order': True, 'enable...",Ansys HFSS,"{'aedt_hfss_capacitance': 0, 'aedt_hfss_induct...",qiskit-metal
1,82.44280,79.19378,2.93820,188.15089,188.15089,333.52997,nH,"Eli Levenson-Falk, PhD",2023-10-25-153123,LFL,USC,Andre Kuo,"{'Cj': 0, 'Lj': '10nH', '_Rj': 0, 'design_name...","{'auto_increase_solution_order': True, 'enable...",Ansys HFSS,"{'aedt_hfss_capacitance': 0, 'aedt_hfss_induct...",qiskit-metal
2,83.76412,80.18130,3.16131,104.35340,104.35340,237.02548,nH,"Eli Levenson-Falk, PhD",2023-09-20-142547,LFL,USC,Andre Kuo,"{'Cj': 0, 'Lj': '10nH', '_Rj': 0, 'design_name...","{'auto_increase_solution_order': True, 'enable...",Ansys HFSS,"{'aedt_hfss_capacitance': 0, 'aedt_hfss_induct...",qiskit-metal
3,103.37057,97.22405,5.77590,174.13928,174.13928,335.31609,nH,"Eli Levenson-Falk, PhD",2023-10-25-153126,LFL,USC,Andre Kuo,"{'Cj': 0, 'Lj': '10nH', '_Rj': 0, 'design_name...","{'auto_increase_solution_order': True, 'enable...",Ansys HFSS,"{'aedt_hfss_capacitance': 0, 'aedt_hfss_induct...",qiskit-metal
4,68.92854,65.68607,2.87375,120.03923,120.03923,240.34085,nH,"Eli Levenson-Falk, PhD",2023-09-20-142547,LFL,USC,Andre Kuo,"{'Cj': 0, 'Lj': '10nH', '_Rj': 0, 'design_name...","{'auto_increase_solution_order': True, 'enable...",Ansys HFSS,"{'aedt_hfss_capacitance': 0, 'aedt_hfss_induct...",qiskit-metal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1929,106.43025,101.53197,4.45645,174.46380,174.46380,340.62919,nH,"Eli Levenson-Falk, PhD",2023-09-20-142547,LFL,USC,Andre Kuo,"{'Cj': 0, 'Lj': '10nH', '_Rj': 0, 'design_name...","{'auto_increase_solution_order': True, 'enable...",Ansys HFSS,"{'aedt_hfss_capacitance': 0, 'aedt_hfss_induct...",qiskit-metal
1930,121.10943,112.62570,7.95178,187.43537,187.43537,367.34003,nH,"Eli Levenson-Falk, PhD",2023-09-20-142549,LFL,USC,Andre Kuo,"{'Cj': 0, 'Lj': '10nH', '_Rj': 0, 'design_name...","{'auto_increase_solution_order': True, 'enable...",Ansys HFSS,"{'aedt_hfss_capacitance': 0, 'aedt_hfss_induct...",qiskit-metal
1931,144.56289,136.36810,7.65968,172.14561,172.14561,372.39970,nH,"Eli Levenson-Falk, PhD",2023-10-25-153123,LFL,USC,Andre Kuo,"{'Cj': 0, 'Lj': '10nH', '_Rj': 0, 'design_name...","{'auto_increase_solution_order': True, 'enable...",Ansys HFSS,"{'aedt_hfss_capacitance': 0, 'aedt_hfss_induct...",qiskit-metal
1932,68.76413,65.78116,2.48795,56.75230,56.75230,166.57383,nH,"Eli Levenson-Falk, PhD",2023-09-20-142547,LFL,USC,Andre Kuo,"{'Cj': 0, 'Lj': '10nH', '_Rj': 0, 'design_name...","{'auto_increase_solution_order': True, 'enable...",Ansys HFSS,"{'aedt_hfss_capacitance': 0, 'aedt_hfss_induct...",qiskit-metal


In [74]:
from squadds import Analyzer

In [75]:
analyzer = Analyzer(db)

In [76]:
analyzer.target_param_keys()

['qubit_frequency_GHz', 'anharmonicity_MHz']

In [77]:
analyzer.__supported_metrics__

['Euclidean', 'Manhattan', 'Chebyshev', 'Weighted Euclidean', 'Custom']

In [78]:
target_params={"qubit_frequency_GHz": 4, "anharmonicity_MHz": -200}

In [80]:
results = analyzer.find_closest(target_params=target_params,
                                       num_top=3,
                                       metric="Euclidean",
                                       display=True)
results

,claw_to_claw,claw_to_ground,cross_to_claw,cross_to_cross,cross_to_ground,ground_to_ground,units,PI,date_created,group,...,renderer_options,setup,simulator,design_options,design_tool,EC,EJ,EJEC,qubit_frequency_GHz,anharmonicity_MHz
643,106.91739,101.13161,5.25204,102.49025,102.49025,255.94708,nH,"Eli Levenson-Falk, PhD",2023-09-20-142547,LFL,...,"{'Cj': 0, 'Lj': '10nH', '_Rj': 0, 'design_name...","{'auto_increase_solution_order': True, 'enable...",Ansys HFSS,"{'aedt_hfss_capacitance': 0, 'aedt_hfss_induct...",qiskit-metal,0.179783,12.278081,68.293902,4.013772,-201.551532
1862,80.01554,76.72741,2.89095,104.64079,104.64079,233.88902,nH,"Eli Levenson-Falk, PhD",2023-09-20-142547,LFL,...,"{'Cj': 0, 'Lj': '10nH', '_Rj': 0, 'design_name...","{'auto_increase_solution_order': True, 'enable...",Ansys HFSS,"{'aedt_hfss_capacitance': 0, 'aedt_hfss_induct...",qiskit-metal,0.180135,12.278081,68.160442,4.017505,-201.973598
1714,76.27207,73.26136,2.62986,104.89818,104.89818,230.69451,nH,"Eli Levenson-Falk, PhD",2023-09-20-142547,LFL,...,"{'Cj': 0, 'Lj': '10nH', '_Rj': 0, 'design_name...","{'auto_increase_solution_order': True, 'enable...",Ansys HFSS,"{'aedt_hfss_capacitance': 0, 'aedt_hfss_induct...",qiskit-metal,0.180141,12.278081,68.158097,4.017570,-201.981031


### Querying for a target cavity design

In [55]:
db.unselect_all()

In [56]:
db.select_system("cavity_claw")

In [57]:
db.select_cavity_claw("RouteMeander")

In [58]:
db.select_coupler("CLT")

In [59]:
db.show_selections()

Selected component:  cavity_claw
Selected component name:  RouteMeander
Selected data type:  eigenmode
Selected system:  cavity_claw
Selected coupler:  CLT


In [60]:
df = db.selected_system_df()

In [61]:
df

,cavity_frequency,kappa,units,PI,date_created,group,institution,uploader,setup,simulator,coupler_type,design_options,design_tool,resonator_type
0,5.353550e+09,161106.598429,Hz,"Eli Levenson-Falk, PhD",2023-12-01-170608,LFL,USC,Andre Kuo,"{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS,CLT,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter
1,8.399241e+09,268412.116632,Hz,"Eli Levenson-Falk, PhD",2023-12-04-124953,LFL,USC,Andre Kuo,"{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS,CLT,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter
2,8.694845e+09,255873.654612,Hz,"Eli Levenson-Falk, PhD",2023-12-09-204334,LFL,USC,Andre Kuo,"{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS,CLT,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter
3,6.616574e+09,30459.761161,Hz,"Eli Levenson-Falk, PhD",2023-12-08-173545,LFL,USC,Andre Kuo,"{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS,CLT,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter
4,7.986835e+09,208304.221064,Hz,"Eli Levenson-Falk, PhD",2023-12-09-204334,LFL,USC,Andre Kuo,"{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS,CLT,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,4.949469e+09,126438.881378,Hz,"Eli Levenson-Falk, PhD",2023-12-01-170608,LFL,USC,Andre Kuo,"{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS,CLT,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter
230,8.805442e+09,291439.656224,Hz,"Eli Levenson-Falk, PhD",2023-12-04-124953,LFL,USC,Andre Kuo,"{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS,CLT,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter
231,6.597444e+09,587144.918000,Hz,"Eli Levenson-Falk, PhD",2023-12-06-224829,LFL,USC,Andre Kuo,"{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS,CLT,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter
232,8.116894e+09,209744.544864,Hz,"Eli Levenson-Falk, PhD",2023-12-09-204334,LFL,USC,Andre Kuo,"{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS,CLT,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter


In [62]:
db.target_param_keys

['cavity_frequency', 'kappa']

In [63]:
analyzer = Analyzer(db)

In [64]:
target_params = {"cavity_frequency_GHz": 6.9,
                "kappa_kHz": 120,
                "resonator_type":"quarter"}

In [69]:
results = analyzer.find_closest(target_params=target_params,
                                       num_top=3,
                                       metric="Euclidean",
                                       display=True)
results

,cavity_frequency_GHz,kappa_kHz,units,PI,date_created,group,institution,uploader,setup,simulator,coupler_type,design_options,design_tool,resonator_type
190,6.371017,121.824395,Hz,"Eli Levenson-Falk, PhD",2023-11-30-214122,LFL,USC,Andre Kuo,"{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS,CLT,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter
123,6.339319,121.639168,Hz,"Eli Levenson-Falk, PhD",2023-11-30-214122,LFL,USC,Andre Kuo,"{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS,CLT,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter
94,6.664178,129.189282,Hz,"Eli Levenson-Falk, PhD",2023-11-30-214122,LFL,USC,Andre Kuo,"{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS,CLT,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter


In [ ]:
# Set up the weights
analyzer.metric_weights = {"cavity_frequency": 1, "kappa": 2}

In [ ]:
results = analyzer.find_closest(target_params=target_params,
                                       num_top=3,
                                       metric="Weighted Euclidean",
                                       display=True)
results

### Querying for a target qubit-cavity design

In [95]:
db.select_system(["qubit","cavity_claw"])

In [96]:
db.select_qubit("TransmonCross")
db.select_cavity_claw("RouteMeander")
db.select_coupler("CLT")

In [106]:
db.show_selections()

Selected qubit:  TransmonCross
Selected cavity:  RouteMeander
Selected coupler:  CLT
Selected system:  ['qubit', 'cavity_claw']


In [98]:
db.selected_df

,claw_to_claw,claw_to_ground,cross_to_claw,cross_to_cross,cross_to_ground,ground_to_ground,units_qubit,PI_qubit,date_created_qubit,group_qubit,...,group_cavity_claw,institution_cavity_claw,uploader_cavity_claw,setup_cavity_claw,simulator_cavity_claw,coupler_type,design_options_cavity_claw,design_tool_cavity_claw,resonator_type,design_options
0,94.97421,90.86585,3.73363,158.40783,158.40783,311.25590,nH,"Eli Levenson-Falk, PhD",2023-09-20-142547,LFL,...,LFL,USC,Andre Kuo,"{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS,CLT,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter,{'cavity_claw_options': {'coupling_type': 'CLT...
1,94.97421,90.86585,3.73363,158.40783,158.40783,311.25590,nH,"Eli Levenson-Falk, PhD",2023-09-20-142547,LFL,...,LFL,USC,Andre Kuo,"{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS,CLT,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter,{'cavity_claw_options': {'coupling_type': 'CLT...
2,94.97421,90.86585,3.73363,158.40783,158.40783,311.25590,nH,"Eli Levenson-Falk, PhD",2023-09-20-142547,LFL,...,LFL,USC,Andre Kuo,"{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS,CLT,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter,{'cavity_claw_options': {'coupling_type': 'CLT...
3,94.97421,90.86585,3.73363,158.40783,158.40783,311.25590,nH,"Eli Levenson-Falk, PhD",2023-09-20-142547,LFL,...,LFL,USC,Andre Kuo,"{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS,CLT,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter,{'cavity_claw_options': {'coupling_type': 'CLT...
4,94.97421,90.86585,3.73363,158.40783,158.40783,311.25590,nH,"Eli Levenson-Falk, PhD",2023-09-20-142547,LFL,...,LFL,USC,Andre Kuo,"{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS,CLT,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter,{'cavity_claw_options': {'coupling_type': 'CLT...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11599,183.80802,168.04023,15.11184,214.45993,214.45993,454.60312,nH,"Eli Levenson-Falk, PhD",2023-09-20-142547,LFL,...,LFL,USC,Andre Kuo,"{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS,CLT,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter,{'cavity_claw_options': {'coupling_type': 'CLT...
11600,183.80802,168.04023,15.11184,214.45993,214.45993,454.60312,nH,"Eli Levenson-Falk, PhD",2023-09-20-142547,LFL,...,LFL,USC,Andre Kuo,"{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS,CLT,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter,{'cavity_claw_options': {'coupling_type': 'CLT...
11601,183.80802,168.04023,15.11184,214.45993,214.45993,454.60312,nH,"Eli Levenson-Falk, PhD",2023-09-20-142547,LFL,...,LFL,USC,Andre Kuo,"{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS,CLT,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter,{'cavity_claw_options': {'coupling_type': 'CLT...
11602,183.80802,168.04023,15.11184,214.45993,214.45993,454.60312,nH,"Eli Levenson-Falk, PhD",2023-09-20-142547,LFL,...,LFL,USC,Andre Kuo,"{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS,CLT,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter,{'cavity_claw_options': {'coupling_type': 'CLT...


In [99]:
merged_df = db.selected_system_df()

In [100]:
merged_df

,claw_to_claw,claw_to_ground,cross_to_claw,cross_to_cross,cross_to_ground,ground_to_ground,units_qubit,PI_qubit,date_created_qubit,group_qubit,...,group_cavity_claw,institution_cavity_claw,uploader_cavity_claw,setup_cavity_claw,simulator_cavity_claw,coupler_type,design_options_cavity_claw,design_tool_cavity_claw,resonator_type,design_options
0,94.97421,90.86585,3.73363,158.40783,158.40783,311.25590,nH,"Eli Levenson-Falk, PhD",2023-09-20-142547,LFL,...,LFL,USC,Andre Kuo,"{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS,CLT,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter,{'cavity_claw_options': {'coupling_type': 'CLT...
1,94.97421,90.86585,3.73363,158.40783,158.40783,311.25590,nH,"Eli Levenson-Falk, PhD",2023-09-20-142547,LFL,...,LFL,USC,Andre Kuo,"{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS,CLT,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter,{'cavity_claw_options': {'coupling_type': 'CLT...
2,94.97421,90.86585,3.73363,158.40783,158.40783,311.25590,nH,"Eli Levenson-Falk, PhD",2023-09-20-142547,LFL,...,LFL,USC,Andre Kuo,"{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS,CLT,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter,{'cavity_claw_options': {'coupling_type': 'CLT...
3,94.97421,90.86585,3.73363,158.40783,158.40783,311.25590,nH,"Eli Levenson-Falk, PhD",2023-09-20-142547,LFL,...,LFL,USC,Andre Kuo,"{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS,CLT,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter,{'cavity_claw_options': {'coupling_type': 'CLT...
4,94.97421,90.86585,3.73363,158.40783,158.40783,311.25590,nH,"Eli Levenson-Falk, PhD",2023-09-20-142547,LFL,...,LFL,USC,Andre Kuo,"{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS,CLT,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter,{'cavity_claw_options': {'coupling_type': 'CLT...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11599,183.80802,168.04023,15.11184,214.45993,214.45993,454.60312,nH,"Eli Levenson-Falk, PhD",2023-09-20-142547,LFL,...,LFL,USC,Andre Kuo,"{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS,CLT,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter,{'cavity_claw_options': {'coupling_type': 'CLT...
11600,183.80802,168.04023,15.11184,214.45993,214.45993,454.60312,nH,"Eli Levenson-Falk, PhD",2023-09-20-142547,LFL,...,LFL,USC,Andre Kuo,"{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS,CLT,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter,{'cavity_claw_options': {'coupling_type': 'CLT...
11601,183.80802,168.04023,15.11184,214.45993,214.45993,454.60312,nH,"Eli Levenson-Falk, PhD",2023-09-20-142547,LFL,...,LFL,USC,Andre Kuo,"{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS,CLT,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter,{'cavity_claw_options': {'coupling_type': 'CLT...
11602,183.80802,168.04023,15.11184,214.45993,214.45993,454.60312,nH,"Eli Levenson-Falk, PhD",2023-09-20-142547,LFL,...,LFL,USC,Andre Kuo,"{'basis_order': 1, 'max_delta_f': 0.05, 'max_p...",Ansys HFSS,CLT,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter,{'cavity_claw_options': {'coupling_type': 'CLT...


In [116]:
analyzer = Analyzer(db)

In [117]:
db.selected_system

['qubit', 'cavity_claw']

In [118]:
analzyer.selected_system

['qubit', 'cavity_claw']

In [119]:
analzyer.__supported_metrics__

['Euclidean', 'Manhattan', 'Chebyshev', 'Weighted Euclidean', 'Custom']

In [132]:
target_params = {
                "qubit_frequency_GHz": 4,
                "cavity_frequency_GHz": 6.7,
                "kappa_kHz": 120,
                "resonator_type":"quarter",
                "anharmonicity_MHz": -200,
                "g_MHz": 70}

In [136]:
results = analyzer.find_closest(target_params=target_params,
                                       num_top=3,
                                       metric="Euclidean",
                                       display=True)
results

   claw_to_claw  claw_to_ground  cross_to_claw  cross_to_cross  \
0      94.97421        90.86585        3.73363       158.40783   
1      94.97421        90.86585        3.73363       158.40783   
2      94.97421        90.86585        3.73363       158.40783   
3      94.97421        90.86585        3.73363       158.40783   
4      94.97421        90.86585        3.73363       158.40783   

   cross_to_ground  ground_to_ground units_qubit                PI_qubit  \
0        158.40783          311.2559          nH  Eli Levenson-Falk, PhD   
1        158.40783          311.2559          nH  Eli Levenson-Falk, PhD   
2        158.40783          311.2559          nH  Eli Levenson-Falk, PhD   
3        158.40783          311.2559          nH  Eli Levenson-Falk, PhD   
4        158.40783          311.2559          nH  Eli Levenson-Falk, PhD   

  date_created_qubit group_qubit  ...  \
0  2023-09-20-142547         LFL  ...   
1  2023-09-20-142547         LFL  ...   
2  2023-09-20-142547   

,claw_to_claw,claw_to_ground,cross_to_claw,cross_to_cross,cross_to_ground,ground_to_ground,units_qubit,PI_qubit,date_created_qubit,group_qubit,...,design_options_cavity_claw,design_tool_cavity_claw,resonator_type,design_options,EC,EJ,EJEC,qubit_frequency_GHz,anharmonicity_MHz,g_MHz
1137,80.47874,75.62952,4.39079,103.02045,103.02045,230.92357,nH,"Eli Levenson-Falk, PhD",2023-09-20-142548,LFL,...,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter,{'cavity_claw_options': {'coupling_type': 'CLT...,0.180337,12.278081,68.084062,4.019646,-202.215941,0.010300
1125,83.76412,80.18130,3.16131,104.35340,104.35340,237.02548,nH,"Eli Levenson-Falk, PhD",2023-09-20-142547,LFL,...,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter,{'cavity_claw_options': {'coupling_type': 'CLT...,0.180164,12.278081,68.149648,4.017807,-202.007813,0.007411
1551,86.95381,83.03407,3.45205,102.62859,102.62859,237.30798,nH,"Eli Levenson-Falk, PhD",2023-10-25-153123,LFL,...,{'claw_opts': {'connection_pads': {'readout': ...,qiskit-metal,quarter,{'cavity_claw_options': {'coupling_type': 'CLT...,0.182599,12.278081,67.240643,4.043524,-204.930795,0.008174


In [131]:
results["cavity_frequency_GHz"]

1125    6.664178
1137    6.664178
1551    6.664178
Name: cavity_frequency_GHz, dtype: float64

In [103]:
analyzer.df

,claw_to_claw,claw_to_ground,cross_to_claw,cross_to_cross,cross_to_ground,ground_to_ground,units,PI,date_created,group,...,renderer_options,setup,simulator,design_options,design_tool,EC,EJ,EJEC,qubit_frequency_GHz,anharmonicity_MHz
0,94.97421,90.86585,3.73363,158.40783,158.40783,311.25590,nH,"Eli Levenson-Falk, PhD",2023-09-20-142547,LFL,...,"{'Cj': 0, 'Lj': '10nH', '_Rj': 0, 'design_name...","{'auto_increase_solution_order': True, 'enable...",Ansys HFSS,"{'aedt_hfss_capacitance': 0, 'aedt_hfss_induct...",qiskit-metal,0.119465,12.278081,102.775549,3.301482,-130.685735
1,82.44280,79.19378,2.93820,188.15089,188.15089,333.52997,nH,"Eli Levenson-Falk, PhD",2023-10-25-153123,LFL,...,"{'Cj': 0, 'Lj': '10nH', '_Rj': 0, 'design_name...","{'auto_increase_solution_order': True, 'enable...",Ansys HFSS,"{'aedt_hfss_capacitance': 0, 'aedt_hfss_induct...",qiskit-metal,0.101368,12.278081,121.124394,3.050502,-109.996600
2,83.76412,80.18130,3.16131,104.35340,104.35340,237.02548,nH,"Eli Levenson-Falk, PhD",2023-09-20-142547,LFL,...,"{'Cj': 0, 'Lj': '10nH', '_Rj': 0, 'design_name...","{'auto_increase_solution_order': True, 'enable...",Ansys HFSS,"{'aedt_hfss_capacitance': 0, 'aedt_hfss_induct...",qiskit-metal,0.180164,12.278081,68.149648,4.017807,-202.007813
3,103.37057,97.22405,5.77590,174.13928,174.13928,335.31609,nH,"Eli Levenson-Falk, PhD",2023-10-25-153126,LFL,...,"{'Cj': 0, 'Lj': '10nH', '_Rj': 0, 'design_name...","{'auto_increase_solution_order': True, 'enable...",Ansys HFSS,"{'aedt_hfss_capacitance': 0, 'aedt_hfss_induct...",qiskit-metal,0.107663,12.278081,114.041661,3.140368,-117.162843
4,68.92854,65.68607,2.87375,120.03923,120.03923,240.34085,nH,"Eli Levenson-Falk, PhD",2023-09-20-142547,LFL,...,"{'Cj': 0, 'Lj': '10nH', '_Rj': 0, 'design_name...","{'auto_increase_solution_order': True, 'enable...",Ansys HFSS,"{'aedt_hfss_capacitance': 0, 'aedt_hfss_induct...",qiskit-metal,0.157593,12.278081,77.910048,3.769716,-175.144766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1929,106.43025,101.53197,4.45645,174.46380,174.46380,340.62919,nH,"Eli Levenson-Falk, PhD",2023-09-20-142547,LFL,...,"{'Cj': 0, 'Lj': '10nH', '_Rj': 0, 'design_name...","{'auto_increase_solution_order': True, 'enable...",Ansys HFSS,"{'aedt_hfss_capacitance': 0, 'aedt_hfss_induct...",qiskit-metal,0.108262,12.278081,113.411011,3.148765,-117.846055
1930,121.10943,112.62570,7.95178,187.43537,187.43537,367.34003,nH,"Eli Levenson-Falk, PhD",2023-09-20-142549,LFL,...,"{'Cj': 0, 'Lj': '10nH', '_Rj': 0, 'design_name...","{'auto_increase_solution_order': True, 'enable...",Ansys HFSS,"{'aedt_hfss_capacitance': 0, 'aedt_hfss_induct...",qiskit-metal,0.099138,12.278081,123.848778,3.017954,-107.466387
1931,144.56289,136.36810,7.65968,172.14561,172.14561,372.39970,nH,"Eli Levenson-Falk, PhD",2023-10-25-153123,LFL,...,"{'Cj': 0, 'Lj': '10nH', '_Rj': 0, 'design_name...","{'auto_increase_solution_order': True, 'enable...",Ansys HFSS,"{'aedt_hfss_capacitance': 0, 'aedt_hfss_induct...",qiskit-metal,0.107729,12.278081,113.972006,3.141292,-117.237917
1932,68.76413,65.78116,2.48795,56.75230,56.75230,166.57383,nH,"Eli Levenson-Falk, PhD",2023-09-20-142547,LFL,...,"{'Cj': 0, 'Lj': '10nH', '_Rj': 0, 'design_name...","{'auto_increase_solution_order': True, 'enable...",Ansys HFSS,"{'aedt_hfss_capacitance': 0, 'aedt_hfss_induct...",qiskit-metal,0.326978,12.278081,37.550231,5.317759,-388.102009


### Simulating the "best-guess" design

## Next Steps...

Let's do a quick recap of what we have learned in this tutorial:

- We have created a HuggingFace account and got an API key

- We have used the SQuADDS API to query for closest and "best-guess" interpolated device designs for our chosen Hamiltonian parameters

- We have simulated the "best-guess" design using an EM solver tool

In the next [tutorial](https://sadmanahmedshanto.com/SQuADDS/tutorials/Tutorial-2_Contributing_to_SQuADDS.html), we will learn how to contribute to the SQuADDS Database.

test

## License
<div style='width: 100%; background-color:#d5d9e0;padding-left: 10px; padding-bottom: 10px; padding-right: 10px; padding-top: 5px'>
    <h3>This code is a part of SQuADDS</h3>
    <p>Developed by Sadman Ahmed Shanto</p>
    <p>&copy; Copyright Sadman Ahmed Shanto & Eli Levenson-Falk 2023.</p>
    <p>This code is licensed under the MIT License. You may<br>obtain a copy of this license in the LICENSE.txt file in the root directory<br> of this source tree.</p>
    <p>Any modifications or derivative works of this code must retain this<br>copyright notice, and modified files need to carry a notice indicating<br>that they have been altered from the originals.</p>
</div>
